In [2]:
# stock trading algorithm that consumes a date and a ticker symbol and utilizes yahoo
# finance data to produce graphs and data tables to present needed info for an investor
# and suggestions on stock outlook based on produced data



# importing stock data. As well as other financial metrics
import pandas_datareader
import pandas_datareader.data as web
import datetime
from pandas_datareader.data import Options
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import quandl
import sys 
import yfinance as yf
import yahoo_fin.stock_info as si
from yahoo_earnings_calendar import YahooEarningsCalendar
import dateutil.parser



# current date / latest date that stocks should be imported on

while True:
    try: 
        year = int(input("enter current year (yyyy): "))
    except ValueError:
        print("Invalid input")
        
        continue
    else:
        break
        
if year<2000 or year>2022:
    print("Invalid year")
    my_year = "invalid"
    
else:
    my_year=str(year)
 
    
while True:
    try:
        month = int(input("enter current month (mm): "))
    except ValueError:
        print("Invalid input")
        
        continue
    else:
        break
if month< 1 or month > 12:
    print("Invalid month")
    my_month = "invalid"
    
else:
    my_month = str(month)


while True:
    try: 
        day = int(input("enter current day (dd): "))
    except ValueError:
        print("Invalid input")
        
        continue
    else:
        break
if day < 1 or day > 31:
    print("Invalid day")
    my_day = "invalid"
    
else:
    my_day = str(day)
        

my_date = str(my_year +'-'+ my_month +'-'+ my_day)
print(my_date)
my_date_start = str(str(int(my_year)-10) + '-'+my_month+'-'+my_day) #start date is 10 years before date entered


if my_year == "invalid" or my_month == "invalid" or my_day == "invalid":
        sys.exit("Invalid date. Cannot run application")

        

# current company / stock that the user wishes to evaluate 
stock_name = str(input("Ticker symbol is: "))

# shortcuts for stock metrics
stock = yf.Ticker(stock_name)
hist = stock.history(period="10y")
market_price = stock.info["regularMarketPrice"]
annual_high = stock.info["fiftyTwoWeekHigh"]
annual_low = stock.info["fiftyTwoWeekLow"]
earnings = stock.earnings['Earnings']



# Heading for stock report 

print(stock_name + ": Stock Fundamentals Report For Year " + my_year + "\n")
print(stock.info['longBusinessSummary'])



# Prints major holders of stock and stock recommendations from institutions 

print(str(stock.major_holders) + "\n")


recommendation_series = stock.recommendations['To Grade']
recommendation_count = recommendation_series.value_counts()
recommendation_dictionary = dict(recommendation_count)
recommendation_buy = recommendation_dictionary['Buy']
recommendation_hold = recommendation_dictionary['Hold']
recommendation_sell = recommendation_dictionary['Sell']

print("'Buy' count = " + str(recommendation_buy))
print("'Hold' count = " + str(recommendation_hold))
print("'Sell' count = " + str(recommendation_sell))

labels = 'Buy','Hold','Sell'
sizes = [recommendation_buy,recommendation_hold,recommendation_sell]
explode = [0.1,0,0]

fig1, ax1 = plt.subplots()
ax1.pie(sizes, explode = explode, labels = labels, autopct='%1.1f%%',shadow=True,startangle=90)
ax1.axis('equal')
plt.show()



# Different stock metrics to see profitability and value of stock

# Market cap 
market_cap = stock.info['marketCap']

print("Market cap = " + str("{:,}".format(market_cap)))

def market_cap_ranking(market_cap):
    if market_cap <= 50000000:
        return("Nano-cap stock")
               
    elif market_cap > 50000000 and market_cap <= 300000000:
        return("Micro-cap stock")
    
    elif market_cap > 300000000 and market_cap <= 2000000000:
        return("Small-cap stock")
    
    elif market_cap > 2000000000 and market_cap <= 10000000000:
        return("Mid-cap stock")
    
    elif market_cap > 10000000000 and market_cap <= 200000000000:
        return("Large-cap stock")
    
    else:
        return("Mega-cap stock")

print(market_cap_ranking(market_cap))

def ten_bagger(market_cap):
    return("market cap if stock were to become ten-bagger = " + str("{:,}".format(market_cap * 10))
          + " = " + market_cap_ranking(market_cap * 10))

def hundred_bagger(market_cap):
    return("market cap if stock were to become hundred-bagger = "+ str(("{:,}".format(market_cap * 100)))
          + " = " + market_cap_ranking(market_cap * 100))

print(ten_bagger(market_cap))
print(hundred_bagger(market_cap) + "\n")



# basic measure to see current stock price in comparison to short term highs/lows
# percent gain to reach 52 week highs
percent_to_high = round((((annual_high / market_price)-1) * 100),2)
percent_to_low = round(((1 - (annual_low / market_price)) * 100),2)

def annual_metrics(market_price, annual_high, annual_low):
    if market_price >= annual_high:
        return ("Stock is currently at / over yearly highs")

    elif abs(market_price - annual_high) > abs(market_price - annual_low):
        return("Low price in comparison to 52 week metrics. \n current jump needed to hit 52 week highs: "
           + str(percent_to_high) + "%" + "\n current fall needed to hit 52 week lows: " + str(percent_to_low)
              + "%")
    
    elif abs(market_price - annual_high) < abs(market_price - annual_low):
        return("High price in comparison to 52 week metrics. \n current jump needed to hit 52 week highs: "
              + str(percent_to_high) + "%" + "\n current fall needed to hit 52 week lows: " + str(percent_to_low)
              + "%")
    

print(annual_metrics(market_price, annual_high, annual_low) + "\n")
    
    
    
    
    
# ROE and ROA 

stats = si.get_stats(stock_name)

roe = (stats[stats.Attribute.str.contains("Return on Equity")])['Value']

array_roe = np.array(roe)
ser_roe = pd.Series(array_roe)


print(stock_name + " has a return on equity in last 12 months of " + str(ser_roe[0]))



roa = (stats[stats.Attribute.str.contains("Return on Assets")])['Value']

array_roa = np.array(roa)
ser_roa = pd.Series(array_roa)

print(stock_name + " has a return on assets in last 12 months of " + str(ser_roa[0] + "\n"))







# find a way to print out earnings and create a graph that has lines projecting current
# earnings trend and then lines projecting estimated earnings 


# putting revenue values into pandas array and then calling on each quarter individually
data = np.array(stock.quarterly_earnings['Revenue'])
ser = pd.Series(data)
first_quarter = ser[0]
second_quarter = ser[1]
third_quarter = ser[2]
fourth_quarter = ser[3]



first_change = (1-(first_quarter / second_quarter)) + 1 
second_change = (1-(second_quarter / third_quarter)) + 1
third_change = (1-(third_quarter / fourth_quarter)) + 1

def quarterly_change(first_change, second_change, third_change):
    if abs(first_change-1) > abs((((second_change + third_change)/2)-1)*2):
        return("Anomaly in first/second quarters\n"
               + str('{:,.2f}'.format((((second_change+third_change)/2)*fourth_quarter))) +
               " is the estimated revenue for the next quarter \n compared to " 
              + str('{:,.2f}'.format(fourth_quarter)) + " this quarter")
    elif abs(second_change-1) > abs((((first_change+third_change)/2)-1)*2):
        return("Anomaly in the second/third quarters \n"
              + str('{:,.2f}'.format((((first_change+third_change)/2)*fourth_quarter))) +
               " is the estimated revenue for the next quarter \n compared to "
              + str('{:,.2f}'.format(fourth_quarter)) + " this quarter")
    elif abs(third_change-1) > abs((((first_change+second_change)/2)-1)*2):
        return("Anomaly in the third/fourth quarters \n"
              + str('{:,.2f}'.format((((first_change+third_change)/2)*fourth_quarter))) +
               " is the estimated revenue for the next quarter \n compared to "
              + str('{:,.2f}'.format(fourth_quarter)) + " this quarter")
    else:
        return("No significant anomalies in revenue \n"
              + str('{:,.2f}'.format((((first_change+second_change+third_change)/3)*fourth_quarter))) +
               " is the estimated revenue for the next quarter \n compared to "
              + str('{:,.2f}'.format(fourth_quarter)) + " this quarter")

print(quarterly_change(first_change, second_change, third_change))

               
    
stock.quarterly_earnings['Revenue'].plot(label='Quarterly Revenues',figsize=(8,5),title='Quarterly Revenues Data')

# Calculates next quarters revenue projections
def quarter_change_estimate(first_change, second_change, third_change):
    if abs(first_change-1) > abs((((second_change + third_change)/2)-1)*2):
        return(((second_change+third_change)/2)*fourth_quarter)
    elif abs(second_change-1) > abs((((first_change+third_change)/2)-1)*2):
        return(((first_change+third_change)/2)*fourth_quarter)
    elif abs(third_change-1) > abs((((first_change+second_change)/2)-1)*2):
        return(((first_change+third_change)/2)*fourth_quarter)
    else:
        return(((first_change+second_change+third_change)/3)*fourth_quarter)
    
    
plt.axhline(y=(quarter_change_estimate(first_change, second_change, third_change)),color='r',
           linestyle='--')
plt.show()



# Plotting and projection annual revenues 
data_annual = np.array(stock.earnings['Revenue'])
ser_annual = pd.Series(data_annual)
first_year = ser_annual[0]
second_year = ser_annual[1]
third_year = ser_annual[2]
fourth_year = ser_annual[3]

first_annual_change = (1-(first_year / second_year)) + 1 
second_annual_change = (1-(second_year / third_year)) + 1
third_annual_change = (1-(third_year / fourth_year)) + 1



def annual_change(first_annual_change, second_annual_change, third_annual_change):
    if abs(first_annual_change-1) > abs((((second_annual_change + third_annual_change)/2)-1)*2):
        return("Anomaly in first/second years\n"
               + str('{:,.2f}'.format((((second_annual_change+third_annual_change)/2)*fourth_year))) +
               " is the estimated revenue for the next year \n compared to " 
              + str('{:,.2f}'.format(fourth_year)) + " this year")
    elif abs(second_annual_change-1) > abs((((first_annual_change+third_annual_change)/2)-1)*2):
        return("Anomaly in the second/third years \n"
              + str('{:,.2f}'.format((((first_annual_change+third_annual_change)/2)*fourth_year))) +
               " is the estimated revenue for the next year \n compared to "
              + str('{:,.2f}'.format(fourth_year)) + " this year")
    elif abs(third_annual_change-1) > abs((((first_annual_change+second_annual_change)/2)-1)*2):
        return("Anomaly in the third/fourth years \n"
              + str('{:,.2f}'.format((((first_annual_change+third_annual_change)/2)*fourth_year))) +
               " is the estimated revenue for the next year \n compared to "
              + str('{:,.2f}'.format(fourth_year)) + " this year")
    else:
        return("No significant anomalies in revenue \n"
              + str('{:,.2f}'.format((((first_annual_change+second_annual_change+third_annual_change)/3)
                                      *fourth_year))) +
               " is the estimated revenue for the next year \n compared to "
              + str('{:,.2f}'.format(fourth_year)) + " this year")

print(annual_change(first_annual_change, second_annual_change, third_annual_change))



stock.earnings['Revenue'].plot(label='Annual Revenues',figsize=(10,5),title='Annual Revenues Data')

# Calculates next quarters revenue projections
def annual_change_estimate(first_annual_change, second_annual_change, third_annual_change):
    if abs(first_annual_change-1) > abs((((second_annual_change + third_annual_change)/2)-1)*2):
        return(((second_annual_change+third_annual_change)/2)*fourth_year)
    elif abs(second_annual_change-1) > abs((((first_annual_change+third_annual_change)/2)-1)*2):
        return(((first_annual_change+third_annual_change)/2)*fourth_year)
    elif abs(third_annual_change-1) > abs((((first_annual_change+second_annual_change)/2)-1)*2):
        return(((first_annual_change+third_annual_change)/2)*fourth_year)
    else:
        return(((first_annual_change+second_annual_change+third_annual_change)/3)*fourth_year)
    
    
plt.axhline(y=(annual_change_estimate(first_annual_change, second_annual_change, third_annual_change)),color='r',
           linestyle='--')
plt.show()


# Measures at the end to conclude whether or not the stock is worth buying based on the short-term indicators

#Metrics 
quarterly_revenue_estimate = quarter_change_estimate(first_change,second_change,third_change)
annual_revenue_estimate = annual_change_estimate(first_annual_change,second_annual_change,third_annual_change)

def annual_revenue_measurements(annual_revenue_estimate,fourth_year):
    if annual_revenue_estimate > fourth_year:
        return("Stock has annual projections that long-term profitability will remain, \n"+
        "therefore investors should buy \n")
    else:
        return("Stock has annual projections that long-term profitability will be lower than before, \n"+
        "therefore investors shouldn't buy \n")
    
def quarterly_revenue_measurements(quarterly_revenue_estimate,fourth_quarter):
    if quarterly_revenue_estimate > fourth_quarter:
        return("Stock has quarterly projections that short-term profitability will remain, \n" +
        "therefore investors should buy \n")
    else:
        return("Stock has annual projections that short-term profitability will be lower than before, \n"+
        "therefore investors shouldn't buy \n")
    
def institution_recommendations(recommendation_buy,recommendation_hold,recommendation_sell):
    if recommendation_buy >= (recommendation_hold + recommendation_sell):
        return("Institutional sentiment regarding stock is that investors should buy \n")
    else:
        return("Institutional sentiment regarding stock is that investors shouldn't buy \n")
    
def annual_price_range(percent_to_high,percent_to_low):
    if percent_to_low >= percent_to_high:
        return("Stock price is high right now compared to 52 week measures \n"+
              "Recommended to wait for stock price to drop before purchasing")
    else:
        return("Stock price is low right now compared to 52 week measures \n"+
              "Recommended to buy soon before stock price goes back up")
    
print(annual_revenue_measurements(annual_revenue_estimate,fourth_year))
print(quarterly_revenue_measurements(quarterly_revenue_estimate,fourth_quarter))
print(institution_recommendations(recommendation_buy,recommendation_hold,recommendation_sell))
print(annual_price_range(percent_to_high,percent_to_low))

enter current year (yyyy): 2020
enter current month (mm): 1
enter current day (dd): 20
2020-1-20
Ticker symbol is: BCE
BCE: Stock Fundamentals Report For Year 2020

BCE Inc., a telecommunications and media company, provides wireless, wireline, Internet, and television (TV) services to residential, business, and wholesale customers in Canada. It operates in three segments: Bell Wireless, Bell Wireline, and Bell Media. The Bell Wireless segment offers wireless voice and data communications products and services. The Bell Wireline segment provides data, including Internet access and Internet protocol television; and local telephone, long distance, and other communications services and products. This segment also buys and sells local telephone, long distance, data, and other services from or to resellers, and other carriers. The Bell Media segment provides conventional TV, specialty TV, pay TV, and streaming services; and digital media, radio broadcasting, out-of-home advertising services.

KeyError: 'Sell'